In [9]:
import pandas as pd
import numpy as np
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR) 

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


In [10]:
data = pd.read_csv('banks.csv', sep='\t')
data.head()

,idx,Score,Text
0,0,Positive,В Альфа-Банке работает замечательная девушка -...
1,1,Negative,Оформляя рассрочку в м. Видео в меге тёплый ст...
2,2,Positive,Очень порадовала оперативность работы в банке....
3,3,Negative,Имела неосторожность оформить потреб. кредит в...
4,4,Negative,Небольшая предыстория: Нашел на сайте MDM банк...


In [16]:
df = pd.read_csv('banks.csv', delimiter='\t')


df['Score'] = LabelEncoder().fit_transform(df['Score']) 
X = df['Text'] 
y = df['Score'] 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [17]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

max_seq_length = 100
X_train_pad = pad_sequences(X_train_seq, maxlen=max_seq_length)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_seq_length)


In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

# Структура модели
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=64, input_length=max_seq_length))
model.add(LSTM(64, return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(32))
model.add(Dense(1, activation='sigmoid'))  # Выводит вероятность класса

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [19]:
model.fit(X_train_pad, y_train, batch_size=32, epochs=5, validation_split=0.1)


Epoch 1/5
315/315 [==============================] - 30s 74ms/step - loss: 0.3075 - accuracy: 0.8650 - val_loss: 0.1973 - val_accuracy: 0.9250
Epoch 2/5
315/315 [==============================] - 20s 62ms/step - loss: 0.1365 - accuracy: 0.9540 - val_loss: 0.1848 - val_accuracy: 0.9312
Epoch 3/5
315/315 [==============================] - 22s 70ms/step - loss: 0.1095 - accuracy: 0.9668 - val_loss: 0.2028 - val_accuracy: 0.9304
Epoch 4/5
315/315 [==============================] - 28s 90ms/step - loss: 0.0774 - accuracy: 0.9768 - val_loss: 0.2193 - val_accuracy: 0.9250
Epoch 5/5
315/315 [==============================] - 21s 67ms/step - loss: 0.0534 - accuracy: 0.9837 - val_loss: 0.2560 - val_accuracy: 0.9205


In [27]:
loss, accuracy = model.evaluate(X_test_pad, y_test)
print(f'точность: {accuracy:.2f}')


88/88 [==============================] - 3s 34ms/step - loss: 0.3101 - accuracy: 0.9050
точность: 0.90


In [28]:
def classify_text(text):
    seq = tokenizer.texts_to_sequences([text])
    padded = pad_sequences(seq, maxlen=max_seq_length)
    prediction = model.predict(padded)
    return 'Positive' if prediction >= 0.1 else 'Negative'

# пример произвольного текста сам написал 
new_text = "я был приятно удивлен отзывчивостью сотрудников банка, когда мне потребовалась помощь с оформлением кредита. Они были внимательны и профессиональны, помогли разъяснить все условия и подобрали оптимальный вариант для меня. Благодаря этому опыту я чувствую себя уверенно и комфортно, обращаясь в этот банк."
print(classify_text(new_text))


1/1 [==============================] - 0s 89ms/step
Positive
